In [ ]:
# import os
# import cv2
# import matplotlib.pyplot as plt

# # Paths to your folders
# dataset_path = "/Volumes/Exodus/Data/ART-Net/"

# train_positive_images_path = os.path.join(dataset_path, "Train/Train_Positive")
# train_negative_images_path = os.path.join(dataset_path, "Train/Train_Negative")
# train_masks_path = os.path.join(dataset_path, "Train/Train_Positive_Tool_Mask")
# train_labels_path = os.path.join(dataset_path, "labels/train")

# test_positive_images_path = os.path.join(dataset_path, "Test/Test_Positive")
# test_negative_images_path = os.path.join(dataset_path, "Test/Test_Negative")
# test_masks_path = os.path.join(dataset_path, "Test/Test_Positive_Tool_Mask")
# test_labels_path = os.path.join(dataset_path, "labels/val")

# os.makedirs(train_labels_path, exist_ok=True)
# os.makedirs(test_labels_path, exist_ok=True)

# # Function to create YOLO annotations from masks
# def create_annotations(images_path, masks_path, labels_path, n = None):
#     if n is not None:
#         image_files = sorted([f for f in os.listdir(images_path) if f.endswith(".png") and not f.startswith("._")])[:n]
#     else:
#         image_files = sorted([f for f in os.listdir(images_path) if f.endswith(".png") and not f.startswith("._")])

#     for image_file in image_files:
#         image_path = os.path.join(images_path, image_file)
#         mask_file = image_file.replace(".png", "_Tool_Mask.png")
#         mask_path = os.path.join(masks_path, mask_file)

#         # print(image_path)
#         # print(mask_path)

#         if os.path.exists(mask_path):
#             # print(f"Processing {image_file}...")
#             # Load the mask image
#             mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
#             contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#             # Open the corresponding label file
#             label_file = os.path.join(labels_path, image_file.replace(".png", ".txt"))
#             with open(label_file, "w") as lf:
#                 for contour in contours:
#                     x, y, w, h = cv2.boundingRect(contour)
#                     # Normalize coordinates
#                     x_center = (x + w / 2) / mask.shape[1]
#                     y_center = (y + h / 2) / mask.shape[0]
#                     width = w / mask.shape[1]
#                     height = h / mask.shape[0]
#                     # Write to label file in YOLO format
#                     lf.write(f"0 {x_center} {y_center} {width} {height}\n")

#                 # # use matplotlib to display the images with bounding box
#                 # image = cv2.imread(image_path)
#                 # for contour in contours:
#                 #     x, y, w, h = cv2.boundingRect(contour)
#                 #     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
#                 # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#                 # plt.axis("off")
#                 # plt.show()

#     print("Annotations created in YOLO format.")

# # Create annotations for training images
# create_annotations(train_positive_images_path, train_masks_path, train_labels_path)
# # Create annotations for test images
# create_annotations(test_positive_images_path, test_masks_path, test_labels_path)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

# Paths to your folders
dataset_path = "/Volumes/Exodus/Data/ART-Net/"

train_positive_images_path = os.path.join(dataset_path, "Train/Train_Positive")
train_negative_images_path = os.path.join(dataset_path, "Train/Train_Negative")
train_masks_path = os.path.join(dataset_path, "Train/Train_Positive_Tool_Mask")
# train_tooltip_masks_path = os.path.join(dataset_path, "Train/Train_Positive_TipPoint")
train_labels_path = os.path.join(dataset_path, "labels/train")

test_positive_images_path = os.path.join(dataset_path, "Test/Test_Positive")
test_negative_images_path = os.path.join(dataset_path, "Test/Test_Negative")
test_masks_path = os.path.join(dataset_path, "Test/Test_Positive_Tool_Mask")
# test_tooltip_masks_path = os.path.join(dataset_path, "Test/Test_Positive_TipPoint")
test_labels_path = os.path.join(dataset_path, "labels/val")

os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)


# Function to create YOLO annotations from masks
def create_annotations(
    images_path, masks_path, tooltip_masks_path, labels_path, n=None
):
    if n is not None:
        image_files = sorted(
            [
                f
                for f in os.listdir(images_path)
                if f.endswith(".png") and not f.startswith("._")
            ]
        )[:n]
    else:
        image_files = sorted(
            [
                f
                for f in os.listdir(images_path)
                if f.endswith(".png") and not f.startswith("._")
            ]
        )

    for image_file in image_files:
        image_path = os.path.join(images_path, image_file)
        mask_file = image_file.replace(".png", "_Tool_Mask.png")
        mask_path = os.path.join(masks_path, mask_file)
        # tooltip_mask_file = image_file.replace(".png", "_TipPoint.png")
        # tooltip_mask_path = os.path.join(tooltip_masks_path, tooltip_mask_file)

        label_file = os.path.join(labels_path, image_file.replace(".png", ".txt"))

        with open(label_file, "w") as lf:
            if os.path.exists(mask_path):
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                contours, _ = cv2.findContours(
                    mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                )
                for contour in contours:
                    contour = contour.squeeze()  # Remove extra dimension
                    if len(contour.shape) == 1:  # Single point, duplicate to make a box
                        contour = np.array([contour, contour])
                    contour = contour.reshape(-1)
                    normalized_contour = []
                    for i in range(0, len(contour), 2):
                        x = contour[i] / mask.shape[1]
                        y = contour[i + 1] / mask.shape[0]
                        normalized_contour.extend([x, y])
                    lf.write(f"0 {' '.join(map(str, normalized_contour))}\n")

            # if os.path.exists(tooltip_mask_path):
            #     tooltip_mask = cv2.imread(tooltip_mask_path, cv2.IMREAD_GRAYSCALE)
            #     tooltip_contours, _ = cv2.findContours(
            #         tooltip_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            #     )
            #     for contour in tooltip_contours:
            #         contour = contour.squeeze()  # Remove extra dimension
            #         if len(contour.shape) == 1:  # Single point, duplicate to make a box
            #             contour = np.array([contour, contour])
            #         contour = contour.reshape(-1)
            #         normalized_contour = []
            #         for i in range(0, len(contour), 2):
            #             x = contour[i] / tooltip_mask.shape[1]
            #             y = contour[i + 1] / tooltip_mask.shape[0]
            #             normalized_contour.extend([x, y])
            #         lf.write(f"1 {' '.join(map(str, normalized_contour))}\n")
    
            # Use matplotlib to draw the images with bounding box
            image = cv2.imread(image_path)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
            # for contour in tooltip_contours:
            #     x, y, w, h = cv2.boundingRect(contour)
            #     cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
                
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.axis("off")

    print("Annotations created in YOLO format.")
            #         contour = contour.reshape(-1)
            #         normalized_contour = []
            #         for i in range(0, len(contour), 2):
            #             x = contour[i] / tooltip_mask.shape[1]
            #             y = contour[i + 1] / tooltip_mask.shape[0]
            #             normalized_contour.extend([x, y])
            #         lf.write(f"1 {' '.join(map(str, normalized_contour))}\n")
    
            # Use matplotlib to draw the images with bounding box
            image = cv2.imread(image_path)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
            # for contour in tooltip_contours:
            #     x, y, w, h = cv2.boundingRect(contour)
            #     cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
                
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.axis("off")

    print("Annotations created in YOLO format.")
            #         contour = contour.reshape(-1)
            #         normalized_contour = []
            #         for i in range(0, len(contour), 2):
            #             x = contour[i] / tooltip_mask.shape[1]
            #             y = contour[i + 1] / tooltip_mask.shape[0]
            #             normalized_contour.extend([x, y])
            #         lf.write(f"1 {' '.join(map(str, normalized_contour))}\n")
    
            # Use matplotlib to draw the images with bounding box
            image = cv2.imread(image_path)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
            # for contour in tooltip_contours:
            #     x, y, w, h = cv2.boundingRect(contour)
            #     cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
                
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.axis("off")

    print("Annotations created in YOLO format.")
            #         contour = contour.reshape(-1)
            #         normalized_contour = []
            #         for i in range(0, len(contour), 2):
            #             x = contour[i] / tooltip_mask.shape[1]
            #             y = contour[i + 1] / tooltip_mask.shape[0]
            #             normalized_contour.extend([x, y])
            #         lf.write(f"1 {' '.join(map(str, normalized_contour))}\n")
    
            # Use matplotlib to draw the images with bounding box
            image = cv2.imread(image_path)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
            # for contour in tooltip_contours:
            #     x, y, w, h = cv2.boundingRect(contour)
            #     cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
                
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.axis("off")

    print("Annotations created in YOLO format.")


# Create annotations for training images
# create_annotations(
#     train_positive_images_path,
#     train_masks_path,
#     train_tooltip_masks_path,
#     train_labels_path,
# )

# # Create annotations for test images
# create_annotations(
#     test_positive_images_path,
#     test_masks_path,
#     test_tooltip_masks_path,
#     test_labels_path,
# )